In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv
/kaggle/input/new-york-city-airbnb-open-data/New_York_City_.png


In [2]:
df = pd.read_csv("/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
df["price"]

0        149
1        225
2        150
3         89
4         80
        ... 
48890     70
48891     40
48892    115
48893     55
48894     90
Name: price, Length: 48895, dtype: int64

In [4]:
use_df = df[["latitude", "longitude", "price", "minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365"]]
use_df.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64749,-73.97237,149,1,9,0.21,6,365
1,40.75362,-73.98377,225,1,45,0.38,2,355
2,40.80902,-73.94190,150,3,0,NaN,1,365
3,40.68514,-73.95976,89,1,270,4.64,1,194
4,40.79851,-73.94399,80,10,9,0.10,1,0


## Q1: Count Missing Values

In [5]:
use_df.isna().sum()

latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

## Q2: Median Value of Minimum_Nights

In [6]:
use_df["minimum_nights"].median()

3.0

## Q3

In [47]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

def rmse(y, y_pred):
    error = y_pred - y
    print(error)
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [22]:
use_df = shuffle(use_df)

In [23]:
X = use_df.drop(["price"], axis=1)
y = use_df["price"]
y = np.log1p(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5)
print(X.shape, X_train.shape, X_val.shape, X_test.shape)

(48895, 7) (29337, 7) (9779, 7) (9779, 7)


In [24]:
X_train["reviews_per_month"] = X_train["reviews_per_month"].fillna(0)
X_val["reviews_per_month"] = X_val["reviews_per_month"].fillna(0)
X_test["reviews_per_month"] = X_test["reviews_per_month"].fillna(0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_val)
rmse = round(math.sqrt(mean_squared_error(y_pred, y_val)),2)
print(rmse)

0.66


In [31]:
w_0, w = train_linear_regression(X_train, y_train)
y_pred = w_0 + X_val.dot(w)
rmse(y_val, y_pred)
np.round(rmse(y_val, y_pred), 2)

0.66

In [32]:
X = use_df.drop(["price"], axis=1)
y = use_df["price"]
y = np.log1p(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5)
print(X.shape, X_train.shape, X_val.shape, X_test.shape)

(48895, 7) (29337, 7) (9779, 7) (9779, 7)


In [33]:
X_train["reviews_per_month"] = X_train["reviews_per_month"].fillna(X_train["reviews_per_month"].mean())
X_val["reviews_per_month"] = X_val["reviews_per_month"].fillna(X_train["reviews_per_month"].mean())
X_test["reviews_per_month"] = X_test["reviews_per_month"].fillna(X_train["reviews_per_month"].mean())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_val)
rmse = round(math.sqrt(mean_squared_error(y_pred, y_val)),2)
print(rmse)

0.65


In [37]:
w_0, w = train_linear_regression(X_train, y_train)
y_pred = w_0 + X_val.dot(w)
rmse(y_val, y_pred)
np.round(rmse(y_val, y_pred), 2)

25934    0.692736
22251    0.133600
20321   -0.807637
8353    -0.241721
14449    0.168120
           ...   
44820    0.466328
8395    -0.439509
13426   -0.306237
14994   -0.731358
44413   -1.588065
Length: 9779, dtype: float64
25934    0.692736
22251    0.133600
20321   -0.807637
8353    -0.241721
14449    0.168120
           ...   
44820    0.466328
8395    -0.439509
13426   -0.306237
14994   -0.731358
44413   -1.588065
Length: 9779, dtype: float64


0.65

better RMSE when missing values filled with 0

## Q4

In [40]:
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [41]:
X = use_df.drop(["price"], axis=1)
y = use_df["price"]
y = np.log1p(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5)
print(X.shape, X_train.shape, X_val.shape, X_test.shape)

(48895, 7) (29337, 7) (9779, 7) (9779, 7)


In [42]:
X_train["reviews_per_month"] = X_train["reviews_per_month"].fillna(0)
X_val["reviews_per_month"] = X_val["reviews_per_month"].fillna(0)
X_test["reviews_per_month"] = X_test["reviews_per_month"].fillna(0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_val)
rmse = round(math.sqrt(mean_squared_error(y_pred, y_val)),2)
print(rmse)

0.64


In [48]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
    y_reg_val = w_0 + X_val.dot(w)
    rmse_val = np.round(rmse(y_val, y_reg_val),2)
    print(r, w_0, rmse_val)

13383    0.484625
36201    0.402052
40936   -0.955371
22854   -1.372228
8532     0.789930
           ...   
280      0.679428
38683   -0.409945
13391    0.328090
22502   -0.474049
40958   -0.044494
Length: 9779, dtype: float64
0 -429.69022634183983 0.64
13383    0.484662
36201    0.402069
40936   -0.955348
22854   -1.372261
8532     0.789949
           ...   
280      0.679404
38683   -0.409950
13391    0.328103
22502   -0.473943
40958   -0.044523
Length: 9779, dtype: float64
1e-06 -429.6392709223564 0.64
13383    0.488272
36201    0.403681
40936   -0.953154
22854   -1.375452
8532     0.791835
           ...   
280      0.677041
38683   -0.410399
13391    0.329358
22502   -0.463567
40958   -0.047396
Length: 9779, dtype: float64
0.0001 -424.65382645395624 0.64
13383    0.517611
36201    0.416790
40936   -0.935315
22854   -1.401393
8532     0.807157
           ...   
280      0.657839
38683   -0.414053
13391    0.339558
22502   -0.379230
40958   -0.070744
Length: 9779, dtype: float64
0.0